In [ ]:
from datasets import load_dataset
from config.finetuning_llama2_tag import config
from utils.data_preprocessor import DataPreprocessorTag
from dotenv import dotenv_values

HF_TOKEN_WRITE = dotenv_values(".env.base")['HF_TOKEN_WRITE']

dataset = load_dataset(config.DATASET_CHEKPOINT) #download_mode="force_redownload"
dataset = dataset[config.TRAIN_LAYER]
dataset = dataset.shuffle(seed=1234)  # Shuffle dataset here

: 

In [35]:
preprocessor = DataPreprocessorTag("meta-llama/Llama-2-7b-chat-hf", "meta-llama/Llama-2-7b-chat-hf", token_llama=HF_TOKEN_WRITE, data =dataset, tagging_string='tag')
display(preprocessor.data)
preprocessor.apply(instruction_on_response_format='')
display(preprocessor.data[0])

MODEL TYPE: llama


Dataset({
    features: ['sentence', 'entities', 'original_text', 'original_id'],
    num_rows: 1520
})

{'sentence': 'The results of the PCR were positive for RNA specific to SARS-CoV-2.', 'entities': [{'id': '9470', 'offsets': [19, 22], 'role': '', 'semantic_type_id': '', 'text': 'PCR', 'type': 'EVENT'}, {'id': '9485', 'offsets': [57, 67], 'role': '', 'semantic_type_id': '', 'text': 'SARS-CoV-2', 'type': 'EVENT'}, {'id': '10107', 'offsets': [57, 67], 'role': '', 'semantic_type_id': 'C5400365', 'text': 'SARS-CoV-2', 'type': 'CLINENTITY'}, {'id': '10347', 'offsets': [28, 36], 'role': '', 'semantic_type_id': '', 'text': 'positive', 'type': 'RML'}], 'original_text': 'A 43-year-old non-diabetic Indian male reported to our outpatient department with chief complaints of cough with expectoration, chest pain, reduced appetite, fever with chills, and night sweats for two weeks. He also complained of breathlessness on exertion and had two episodes of blood in his sputum. The patient explained that the cough was continuous and was relieved after taking cough syrup. He also mentioned that the episod

Map: 100%|██████████| 1520/1520 [00:00<00:00, 6926.78 examples/s] 


{'sentence': 'The results of the PCR were positive for RNA specific to SARS-CoV-2.',
 'entities': [{'id': '9470',
   'offsets': [19, 22],
   'role': '',
   'semantic_type_id': '',
   'text': 'PCR',
   'type': 'EVENT'},
  {'id': '9485',
   'offsets': [57, 67],
   'role': '',
   'semantic_type_id': '',
   'text': 'SARS-CoV-2',
   'type': 'EVENT'},
  {'id': '10107',
   'offsets': [57, 67],
   'role': '',
   'semantic_type_id': 'C5400365',
   'text': 'SARS-CoV-2',
   'type': 'CLINENTITY'},
  {'id': '10347',
   'offsets': [28, 36],
   'role': '',
   'semantic_type_id': '',
   'text': 'positive',
   'type': 'RML'}],
 'original_text': 'A 43-year-old non-diabetic Indian male reported to our outpatient department with chief complaints of cough with expectoration, chest pain, reduced appetite, fever with chills, and night sweats for two weeks. He also complained of breathlessness on exertion and had two episodes of blood in his sputum. The patient explained that the cough was continuous and was 

In [54]:
def from_generativa_to_tag(example, entity_type_filter = False, entity_type='CLINENTITY'):
    to_add = len('<tag></tag>')
    base_sentence = example['sentence']
    entities = example['entities']
    entities = sorted(entities, key=lambda x: x['offsets'][0])
    sentence = ''
    prev_end=0

    # Initialize an empty list to store the filtered data, removing overlapping offsets
    filtered_entities = []
    current_end = -1
    for item in entities:
        start, end = item['offsets']
        if start > current_end:
            filtered_entities.append(item)
            current_end = end
    if entity_type_filter: filtered_entities = [ent for ent in filtered_entities if ent['type'] == entity_type]
    while len(filtered_entities) > 0:
        ent = filtered_entities.pop(0)
        start = ent['offsets'][0]
        end = ent['offsets'][1]
        sentence = sentence + base_sentence[prev_end:start] + '<tag>' + base_sentence[start:end] + '</tag>'
        prev_end = end
        if len(filtered_entities) == 0:
            sentence = sentence + base_sentence[end:]
            break
    if sentence=='':
        sentence = base_sentence
    example['prompt_tag'] = sentence
    return example

dataset = dataset.map(lambda x: from_generativa_to_tag(x, entity_type_filter=False))



Map: 100%|██████████| 1520/1520 [00:00<00:00, 9105.11 examples/s]


['The results of the <tag>PCR</tag> were <tag>positive</tag> for RNA specific to <tag>SARS-CoV-2</tag>.',
 '<tag>Their child</tag> <tag>acquired</tag> <tag>walking</tag> at the age of 14 months.',
 'An abdominal ultrasound <tag>examination</tag> was <tag>reported</tag> as <tag>normal</tag>.',
 'Flow <tag>cytometry</tag> of <tag>the CSF</tag> <tag>showed</tag> that <tag>blast cells</tag> were <tag>positive</tag> for a cluster of differentiation <tag>markers</tag> (CD) (CD34, CD19, CD10, CD22 and <tag>partially positive</tag> for <tag>CD45</tag>) <tag>confirming</tag> <tag>CNS</tag> extramedullary lymphoid blast <tag>infiltration</tag>.',
 'Her <tag>blood</tag> <tag>pressure</tag> was <tag>100/70 mmHg</tag> with a pulse <tag>rate</tag> of <tag>98 beats/min</tag>, respiratory <tag>rate</tag> about <tag>16/min</tag> and oral <tag>temperature</tag> of <tag>37°C</tag>.',
 '<tag>She</tag> had also <tag>positive</tag> <tag>serum</tag> antibody against the Smith antigen and low serum <tag>level